In [1]:
%matplotlib inline

import sys
import numpy as np
from tqdm import tqdm
from threading import Thread
import plotly.graph_objects as go
from multiprocessing import Process

sys.path.append('../library/core')

from problems import *
from view import * 
from create_mov import *

In [2]:
print(os.cpu_count())

16


In [3]:
class PSO:
    def __init__(self, num_particles, inertia=0.9, global_acceleration=0.9, personal_acceleration=0.9):
        self.num_particles = num_particles
        self.inertia = inertia
        self.global_acceleration = global_acceleration
        self.personal_acceleration = personal_acceleration

        self.problem = None
        self.global_best_idx = None
        self.particles = None
    
    def initialize(self, problem):
        self.problem = problem

        #--- Create the initial particle swarm.
        self.particles = np.zeros((self.num_particles, 2, 3)) # NOTE: states[i] = [[position, personal_best, velocity].T]
        initial_positions = problem.max_range_value * np.random.uniform(low=-1, high=1, size=(self.num_particles, 2))
        initial_velocities = 2 * problem.max_range_value * np.random.uniform(low=-1, high=1, size=(self.num_particles, 2))

        self.particles[..., 0] = initial_positions
        self.particles[..., 1] = initial_positions
        self.particles[..., 2] = initial_velocities

        self.global_best_idx = problem.best_index(initial_positions)

    def update(self, problem):
        positions = self.particles[..., 0]
        personal_bests = self.particles[..., 1]
        velocities = self.particles[..., 2]
        global_best_pos = self.particles[self.global_best_idx, ..., 0]

        new_velocities = self.inertia * velocities
        new_velocities += self.global_acceleration * (global_best_pos - positions) * np.random.rand()
        new_velocities += self.personal_acceleration * (personal_bests - positions) * np.random.rand()

        new_positions = positions + new_velocities

        old_and_new_concat_pos = np.stack([positions, new_positions], 2)
        old_and_new_pos = np.transpose(old_and_new_concat_pos, (0, 2, 1))

        new_personal_bests = problem.personal_best(old_and_new_pos)
        new_global_best_idx = problem.best_index(new_personal_bests)

        self.particles[..., 0] = new_positions
        self.particles[..., 1] = new_personal_bests
        self.particles[..., 2] = new_velocities
        self.global_best_idx = new_global_best_idx



In [7]:
prob = FiveWellPotential()

num_p = 50
T = 200

pso = PSO(num_particles=num_p, 
          inertia=0.95, 
          global_acceleration=0.8, 
          personal_acceleration=0.5)

In [8]:

problem_name = prob.problem_name
max_range_val = prob.max_range_value

record = np.zeros((T, num_p, 2))
y_record = np.zeros((T, num_p))

pso.initialize(prob)

for i in tqdm(range(T)):
    record[i] = pso.particles[..., 0]
    y_record[i] = prob.eval(record[i])
    pso.update(prob)
    


100%|██████████| 200/200 [00:00<00:00, 3582.81it/s]


In [9]:

# for i in tqdm(range(T)):
#     drawGriewank_withWire(prob = prob,
#                           x1 = record[i, :, 0],
#                           x2 = record[i, :, 1],
#                           y = y_record[i],
#                           target = f'../../../figures/griewank/griewank_process/{i}_griewank.png',
#                           vis = False)
    
#     if i == 0 or i == int(T/2) or i == T-1:
#         drawGriewank_withWire(prob = prob,
#                               x1 = record[i, :, 0],
#                               x2 = record[i, :, 1],
#                               y = y_record[i],
#                               target = f'../../../figures/griewank/interactive_{i}_griewank.html',
#                               vis = False)

In [10]:
# threads = []
# for i in range(T):
#     thread = Thread(target=drawGriewank_withWire, args=(prob, record[i, :, 0], record[i, :, 1], y_record[i], f'../../../figures/griewank/griewank_process/{i}_griewank.png', False))
#     threads.append(thread)
#     thread.start()

#     if i == 0 or i == int(T/2) or i == T-1:
#         thread = Thread(target=drawGriewank_withWire, args=(prob, record[i, :, 0], record[i, :, 1], y_record[i], f'../../../figures/griewank/interactive_{i}_griewank.html', False))
#         threads.append(thread)
#         thread.start()

# for thread in threads:
#     thread.join()

In [11]:
processes = []
for i in range(T):
    process = Process(target=drawGriewank_withWire, 
                      args=(prob,
                            record[i, :, 0],
                            record[i, :, 1],
                            y_record[i],
                            dict(text = f'{problem_name} T={i}',
                                 font = dict(size=50, color='black'),
                                 x = 0.2,
                                 y = 0.93,
                                 xanchor = 'center'),
                            f'../../../figures/{problem_name}/{problem_name}_process/{i}_{problem_name}.png',
                            False))
    processes.append(process)
    process.start()
    
    if i == 0 or i == int(T/2) or i == T-1:
        process = Process(target=drawGriewank_withWire,
                          args=(prob, 
                                record[i, :, 0], 
                                record[i, :, 1], 
                                y_record[i], 
                                dict(text = f'{problem_name} T={i}',
                                     font = dict(size=26, color='black'),
                                     x = 0.2,
                                     y = 0.93,
                                     xanchor = 'center'),
                                f'../../../figures/{problem_name}/interactive_{i}_{problem_name}.html', 
                                False))
        processes.append(process)
        process.start()

for process in processes:
    process.join()

In [12]:
save_gif(problem_name,
         duration = 150,
         loop = 0)

gif2mp4(f'../../../figures/{problem_name}/{problem_name}.gif')

Moviepy - Building video ../../../figures/fivewellpotential/fivewellpotential.mp4.
Moviepy - Writing video ../../../figures/fivewellpotential/fivewellpotential.mp4



Moviepy - Done !
Moviepy - video ready ../../../figures/fivewellpotential/fivewellpotential.mp4
